In [17]:
# Import the necessary libraries first
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.feature_selection import chi2, r_regression
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import yfinance as yf
import talib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import LSTM,Dropout, BatchNormalization, Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.optimizer_v2.adam import Adam
from keras.regularizers import L1L2

In [18]:
data=yf.download('0700.hk')
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-06-16,0.875,0.925,0.815,0.830,0.802119,2198875000
2004-06-17,0.830,0.875,0.825,0.845,0.816615,419007500
2004-06-18,0.840,0.850,0.790,0.805,0.777959,182990000
2004-06-21,0.820,0.825,0.790,0.800,0.773127,114085000
2004-06-22,0.800,0.800,0.800,0.800,0.773127,0


In [19]:
data['Prediction']=data['Adj Close'].shift(-1)
data.drop('Adj Close',1,inplace=True)
data.head()

/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_39278/1580295549.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop('Adj Close',1,inplace=True)


,Open,High,Low,Close,Volume,Prediction
Date,,,,,,
2004-06-16,0.875,0.925,0.815,0.830,2198875000,0.816615
2004-06-17,0.830,0.875,0.825,0.845,419007500,0.777959
2004-06-18,0.840,0.850,0.790,0.805,182990000,0.773127
2004-06-21,0.820,0.825,0.790,0.800,114085000,0.773127
2004-06-22,0.800,0.800,0.800,0.800,0,0.855272


In [20]:
data['6day MA'] = data['Close'].rolling(window = 6).mean()
data['12day MA'] = data['Close'].rolling(window = 12).mean()
data['RSI'] = talib.RSI(data['Close'].values, timeperiod = 7)
data['%R5'] = talib.WILLR(data['High'].values, data['Low'].values, data['Close'].values, 5)
data['%R10'] = talib.WILLR(data['High'].values, data['Low'].values, data['Close'].values, 10)
data['MI6']=talib.MOM(data['Close'],timeperiod=6)
data['MI12']=talib.MOM(data['Close'],timeperiod=12)
macd, macdsignal, macdhist = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['EMA12']=talib.EMA(data['Close'], timeperiod=12)
data['EMA26']=talib.EMA(data['Close'],timeperiod=26)
data['MACD']=macd
data['TR']=talib.TRANGE(data['High'],data['Low'],data['Close'])
data['OSC6']=talib.CMO(data['Close'], timeperiod=6)
data['OSC12']=talib.CMO(data['Close'], timeperiod=12)
data.head()

,Open,High,Low,Close,Volume,Prediction,6day MA,12day MA,RSI,%R5,%R10,MTM6,MTM12,EMA12,EMA26,MACD,TR,OSC6,OSC12
Date,,,,,,,,,,,,,,,,,,,
2004-06-16,0.875,0.925,0.815,0.830,2198875000,0.816615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-06-17,0.830,0.875,0.825,0.845,419007500,0.777959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.050,NaN,NaN
2004-06-18,0.840,0.850,0.790,0.805,182990000,0.773127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060,NaN,NaN
2004-06-21,0.820,0.825,0.790,0.800,114085000,0.773127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035,NaN,NaN
2004-06-22,0.800,0.800,0.800,0.800,0,0.855272,NaN,NaN,NaN,-92.592599,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN


In [21]:
data.dropna(inplace=True)

In [22]:
X=data.drop('Prediction',1)
Y=data['Prediction'].values.reshape(-1,1)

/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_39278/621122723.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X=data.drop('Prediction',1)


In [23]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestRegressor
#k_features=10 (It will get top 10 features best suited for prediction)
#forward=False (Backward feature selection model)
#verbose=2 (It will show details output as shown below.)
#cv=5 (Kfold cross valiation: it will split the training set in 5 set and 4 will be using for training the model and 1 will using as validation)
#n_jobs=-1 (Number of cores it will use for execution.-1 means it will use all the cores of CPU for execution.)
#scoring='r2'(R-squared is a statistical measure of how close the data are to the fitted regression line)
backwardModel=sfs(RandomForestRegressor(),k_features=10,forward=False,verbose=2,cv=5,n_jobs=-1,scoring='r2')
#We will convert our training data into numpy array. If we will not convert it, model is not able to read some of the column names.
backwardModel.fit(np.array(X),Y.ravel())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed:   26.2s remaining:   10.1s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   35.5s finished

[2021-12-31 22:08:44] Features: 17/10 -- score: 0.13248835096186595[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  17 | elapsed:   23.0s remaining:   12.6s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:   32.0s finished

[2021-12-31 22:09:16] Features: 16/10 -- score: 0.14017480731561302[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:   21.3s remaining:   12.8s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   21.6s finished

[2021-12-31 22:09:37] Features: 15/10 -- score: 0.13406555481775964[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out 

SequentialFeatureSelector(estimator=RandomForestRegressor(), forward=False,
                          k_features=10, n_jobs=-1, scoring='r2', verbose=2)

In [24]:
#Get the column name for the selected feature.
X.columns[list(backwardModel.k_feature_idx_)]

Index(['Open', 'Low', 'Close', '6day MA', 'RSI', '%R5', '%R10', 'EMA12',
       'MACD', 'OSC6'],
      dtype='object')

In [25]:
def trim_dataset(mat, batch_size):
    
    #trims dataset to a size that's divisible by BATCH_SIZE
    no_of_rows_drop = mat.shape[0] % batch_size

    if no_of_rows_drop > 0:
        return mat[:-no_of_rows_drop]
    else:
        return mat
    
def build_timeseries(mat, y_pred):
    
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]

    x = np.zeros((dim_0, TIME_STEPS, dim_1))
    y = np.zeros((dim_0,))

    print("Length of inputs", dim_0)

    for i in range(dim_0):
        x[i] = mat[i:TIME_STEPS+i]
        y[i] = y_pred[TIME_STEPS+i,0]

    print("length of time-series - inputs", x.shape)
    print("length of time-series - outputs", y.shape)

    return x, y

train_cols = selected_features_rfe
params = {
    "batch_size": 20,  # 20<16<10, 25 was a bust
    "epochs": 300,
    "lr": 0.00010000,
    "time_steps": 10
}
TIME_STEPS = params["time_steps"]
BATCH_SIZE = params["batch_size"]
df_train, df_test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=False,random_state=42)
print("Train--Test size", len(df_train), len(df_test))
min_max_scaler = MinMaxScaler()
min_max_scaler_y = MinMaxScaler()
x = df_train.loc[:,train_cols].values
y_train=df_train['Prediction'].values.reshape(-1,1)
y_train=min_max_scaler_y.fit_transform(y_train)
x_train = min_max_scaler.fit_transform(x)
x_test = min_max_scaler.transform(df_test.loc[:,train_cols])
y_test=df_test['Prediction'].values.reshape(-1,1)
y_test=min_max_scaler_y.fit_transform(y_test)
x_t, y_t = build_timeseries(x_train, y_train)
x_t = trim_dataset(x_t, BATCH_SIZE)
y_t = trim_dataset(y_t, BATCH_SIZE).reshape(-1,1)
print(x_t)
print(y_t)
print("Batch trimmed size", x_t.shape, y_t.shape)

NameError: name 'selected_features_rfe' is not defined

In [ ]:
x_temp, y_temp = build_timeseries(x_test,y_test)
x_val, x_test_t = np.split(trim_dataset(x_temp, BATCH_SIZE),2)
y_val, y_test_t = np.split(trim_dataset(y_temp, BATCH_SIZE),2)
print(x_test_t.shape)

In [ ]:
# # Build the LSTM model
adam = Adam(lr=0.001)
reg=L1L2(l1=0.01, l2=0.01)
# BATCH_SIZE=1
# X=data_selected.drop('Prediction',1).values
# sc = MinMaxScaler(feature_range=(-1,1))
# sc_y = MinMaxScaler(feature_range=(-1,1))
# X=sc.fit_transform(X)
# X = X.reshape(X.shape[0], 1, X.shape[1])
# y=data_selected['Prediction'].values.reshape(-1,1)
# y=sc_y.fit_transform(y)
# print(y)
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
model = Sequential()
model.add(LSTM(128, batch_input_shape=(BATCH_SIZE,TIME_STEPS,nof),
                        dropout=0.0, recurrent_dropout=0.0,
                        stateful=True, return_sequences=True,
                        kernel_initializer='random_uniform',
                        bias_regularizer=reg))
model.add(Dropout(0.4))

model.add(LSTM(64, dropout=0.0))
# model.add(Dropout(0.4))
    
model.add(Dense(32))
model.add(Dense(1))
# model.add(LSTM(7,input_shape=(1, X.shape[2]),bias_regularizer=reg,dropout=0.4,unit_forget_bias=False,return_sequences=True))
# model.add(Dense(1))
model.compile(
  loss="mean_squared_error",
  optimizer=adam
)

In [ ]:
model.summary() 

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40, min_delta=0.0001)
history = model.fit(x_t,y_t,epochs=100,batch_size=BATCH_SIZE,validation_data=(trim_dataset(x_val, BATCH_SIZE),
                        trim_dataset(y_val, BATCH_SIZE)),shuffle=False, callbacks=[es])

In [ ]:
y_pred = model.predict(trim_dataset(x_test_t,BATCH_SIZE),batch_size=BATCH_SIZE)
y_pred = y_pred.flatten()
y_test_t = trim_dataset(y_test_t, BATCH_SIZE)
error = mean_squared_error(y_test_t, y_pred)
print("Error is", error, y_pred.shape, y_test_t.shape)

# convert the predicted value to range of real data
y_pred_org = min_max_scaler_y.inverse_transform(y_pred.reshape(-1,1))
# min_max_scaler.inverse_transform(y_pred)
y_test_t_org =min_max_scaler_y.inverse_transform(y_test_t.reshape(-1,1))
# min_max_scaler.inverse_transform(y_test_t)
print(y_pred_org)
print(y_test_t_org)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test_t)), y_test_t_org, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test_t)), y_pred_org, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
plt.plot(y_test_t_org, marker='.', label="true")
plt.plot(y_pred_org, 'r',marker='*', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test_t_org,y_pred_org)

In [ ]:
#model.save('HSBC_FeatureSelected')

In [ ]:
x_unseen.shape

In [ ]:
y_unseen=data_unseen['Prediction'].values.reshape(-1,1)
y_unseen_pred=model.predict(x_unseen,batch_size=20)
y_unseen_pred=min_max_scaler_y.inverse_transform(y_unseen_pred)

In [ ]:
plt.plot(y_unseen, marker='.', label="true")
plt.plot(y_unseen_pred, 'r',marker='*', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
r2_score(data_unseen['Prediction'].values,unseen_pred)